<a href="https://colab.research.google.com/github/malorieiovino/llm-pilot/blob/main/pilot_summary_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Setup

In [2]:
!git clone https://github.com/malorieiovino/llm-pilot.git

Cloning into 'llm-pilot'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 25.53 KiB | 8.51 MiB/s, done.


In [3]:
%cd llm-pilot

/content/llm-pilot


In [4]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re

In [5]:
!pip install --quiet gspread gspread_dataframe --upgrade

In [6]:
def load_data():
    """Load your annotated dataset from Google Sheets"""
    import gspread, pandas as pd
    from google.colab import auth

    # Authenticate
    auth.authenticate_user()

    import google.auth
    creds, _ = google.auth.default()

    gc = gspread.authorize(creds)

    # Open your spreadsheet (replace with your actual sheet name)
    sh = gc.open("pilot_data")  # Change this to your sheet name
    ws = sh.sheet1  # Or specify the worksheet name

    # Get all records and convert to DataFrame
    df = pd.DataFrame(ws.get_all_records())

    return df

## Model Testing

### Summarization Model Initialization



In [7]:
def setup_models():

    models = {}

    # BART (CNN/DailyMail trained - news style)
    models['bart-cnn'] = pipeline('summarization',
                                 model='facebook/bart-large-cnn',
                                 device=0 if torch.cuda.is_available() else -1)

    # BART (XSUM trained - more abstractive)
    models['bart-xsum'] = pipeline('summarization',
                                  model='facebook/bart-large-xsum',
                                  device=0 if torch.cuda.is_available() else -1)

    # T5 (encoder-decoder architecture)
    models['t5-base'] = pipeline('summarization',
                                model='t5-base',
                                device=0 if torch.cuda.is_available() else -1)

    # T5 large (more parameters)
    models['t5-large'] = pipeline('summarization',
                                 model='t5-large',
                                 device=0 if torch.cuda.is_available() else -1)

    # Pegasus (designed for summarization)
    models['pegasus-xsum'] = pipeline('summarization',
                                     model='google/pegasus-xsum',
                                     device=0 if torch.cuda.is_available() else -1)

    # Pegasus CNN/DailyMail
    models['pegasus-cnn'] = pipeline('summarization',
                                    model='google/pegasus-cnn_dailymail',
                                    device=0 if torch.cuda.is_available() else -1)

    # DistilBART (faster, smaller)
    models['distilbart'] = pipeline('summarization',
                                   model='sshleifer/distilbart-cnn-12-6',
                                   device=0 if torch.cuda.is_available() else -1)

    # LED (for longer documents)
    models['led'] = pipeline('summarization',
                            model='allenai/led-base-16384',
                            device=0 if torch.cuda.is_available() else -1)

    return models

In [8]:
#test sets
def create_test_sets(df):
    test_sets = {}

    # Clean samples (no tagged phenomena)
    clean_mask = df['LING_FEATURES'].isna() | (df['LING_FEATURES'] == '') | (df['LING_FEATURES'] == '0')
    test_sets['clean'] = df[clean_mask].copy()

    # Samples with conditionals
    cond_mask = df['LING_FEATURES'].str.contains('cond_', na=False)
    test_sets['conditionals'] = df[cond_mask].copy()

    # Samples with uncertainty
    uncert_mask = df['LING_FEATURES'].str.contains('uncertainty_', na=False)
    test_sets['uncertainty'] = df[uncert_mask].copy()

    # Samples with disfluency
    disfl_mask = df['LING_FEATURES'].str.contains('disfluency_', na=False)
    test_sets['disfluency'] = df[disfl_mask].copy()

    # Multiple phenomena (complex cases)
    multi_mask = df['LING_FEATURES'].str.contains(',', na=False)
    test_sets['multiple'] = df[multi_mask].copy()

    return test_sets

In [9]:
#summary generation
def generate_summaries(test_sets, models, max_length=150):
    """Generate summaries for each test set with each model"""

    results = {}

    for set_name, data in test_sets.items():
        if len(data) == 0:
            continue

        results[set_name] = {}

        # Take a sample if dataset is large (for pilot)
        sample_data = data.head(10) if len(data) > 10 else data

        for model_name, model in models.items():
            print(f"Processing {set_name} with {model_name}...")

            summaries = []

            for idx, row in sample_data.iterrows():
                try:
                    # Clean the text
                    text = str(row['TEXT']).strip()

                    # Generate summary
                    if model_name == 't5':
                        # T5 needs "summarize: " prefix
                        text = "summarize: " + text

                    summary = model(text,
                                  max_length=max_length,
                                  min_length=30,
                                  do_sample=False)

                    summaries.append({
                        'original_text': row['TEXT'],
                        'summary': summary[0]['summary_text'],
                        'linguistic_features': row['LING_FEATURES'],
                        'original_index': idx
                    })

                except Exception as e:
                    print(f"Error processing text {idx}: {e}")
                    continue

            results[set_name][model_name] = summaries

    return results

In [10]:
#quality check
def assess_quality(results):
    """Basic quality assessment of summaries"""

    quality_scores = {}

    for set_name, model_results in results.items():
        quality_scores[set_name] = {}

        for model_name, summaries in model_results.items():
            scores = []

            for summary_data in summaries:
                original = summary_data['original_text']
                summary = summary_data['summary']

                # Basic quality metrics
                score = {
                    'length_ratio': len(summary) / len(original),
                    'contains_key_info': check_key_info_preservation(original, summary),
                    'linguistic_issues': detect_linguistic_issues(summary),
                    'coherence_score': assess_coherence(summary)
                }

                scores.append(score)

            # Average scores for this set/model combination
            if scores:
                avg_scores = {
                    'avg_length_ratio': np.mean([s['length_ratio'] for s in scores]),
                    'key_info_preservation': np.mean([s['contains_key_info'] for s in scores]),
                    'linguistic_issues': np.mean([s['linguistic_issues'] for s in scores]),
                    'coherence': np.mean([s['coherence_score'] for s in scores]),
                    'sample_size': len(scores)
                }
                quality_scores[set_name][model_name] = avg_scores

    return quality_scores

def check_key_info_preservation(original, summary):
    """Check if key information is preserved (basic version)"""
    # Look for important entities/concepts
    important_patterns = [
        r'\b\d{4}\b',  # Years
        r'\$[\d,]+',   # Money amounts
        r'\b[A-Z][a-z]+ [A-Z][a-z]+\b',  # Proper names
        r'\b(contract|agreement|defendant|plaintiff|court)\b'  # Legal terms
    ]

    score = 0
    total_patterns = 0

    for pattern in important_patterns:
        original_matches = set(re.findall(pattern, original, re.IGNORECASE))
        summary_matches = set(re.findall(pattern, summary, re.IGNORECASE))

        if original_matches:
            total_patterns += 1
            # What percentage of original matches are in summary?
            if original_matches:
                preservation = len(original_matches & summary_matches) / len(original_matches)
                score += preservation

    return score / total_patterns if total_patterns > 0 else 1.0


In [11]:
def detect_linguistic_issues(summary):
    """Detect potential linguistic issues in summary"""
    issues = 0

    # Check for incomplete sentences
    if not summary.strip().endswith(('.', '!', '?')):
        issues += 1

    # Check for weird repetition
    words = summary.lower().split()
    if len(words) != len(set(words)) and len(words) > 5:
        repetition_ratio = 1 - (len(set(words)) / len(words))
        if repetition_ratio > 0.3:  # More than 30% repetition
            issues += 1

    # Check for fragments (very basic)
    sentences = summary.split('.')
    short_sentences = [s for s in sentences if len(s.strip().split()) < 3]
    if len(short_sentences) > len(sentences) * 0.5:
        issues += 1

    return issues

def assess_coherence(summary):
    """Basic coherence assessment"""
    sentences = [s.strip() for s in summary.split('.') if s.strip()]

    if len(sentences) < 2:
        return 1.0

    # Very basic coherence: check for discourse markers
    coherence_markers = ['however', 'therefore', 'furthermore', 'additionally',
                        'meanwhile', 'consequently', 'moreover', 'thus']

    has_markers = any(marker in summary.lower() for marker in coherence_markers)

    # Penalize if no coherence markers in multi-sentence summary
    return 1.0 if has_markers or len(sentences) == 1 else 0.7


In [12]:
# Main execution function
def run_pilot_study():
    """Run the complete pilot study"""

    print("Loading data...")
    df = load_data()

    print("Setting up models...")
    models = setup_models()

    print("Creating test sets...")
    test_sets = create_test_sets(df)

    print("Test set sizes:")
    for name, data in test_sets.items():
        print(f"  {name}: {len(data)} samples")

    print("\nGenerating summaries...")
    results = generate_summaries(test_sets, models)

    print("Assessing quality...")
    quality_scores = assess_quality(results)

    return results, quality_scores

# Analysis and visualization
def analyze_results(quality_scores):
    """Analyze and display results"""

    # Convert to DataFrame for easier analysis
    rows = []
    for set_name, models in quality_scores.items():
        for model_name, scores in models.items():
            row = {
                'test_set': set_name,
                'model': model_name,
                **scores
            }
            rows.append(row)

    results_df = pd.DataFrame(rows)

    print("=== PILOT STUDY RESULTS ===\n")

    # Summary by test set
    print("Average scores by linguistic phenomena:")
    summary = results_df.groupby('test_set')[['key_info_preservation', 'linguistic_issues', 'coherence']].mean()
    print(summary.round(3))

    print("\n" + "="*50)

    # Model comparison
    print("\nAverage scores by model:")
    model_summary = results_df.groupby('model')[['key_info_preservation', 'linguistic_issues', 'coherence']].mean()
    print(model_summary.round(3))

    return results_df

In [13]:
if __name__ == "__main__":
    print("Starting pilot study...")

    # Run the pilot study with debugging
    try:
        results, quality_scores = run_pilot_study()
        print(f"\nGot results for {len(results)} test sets")
        print(f"Got quality scores for {len(quality_scores)} test sets")

        # Print some basic info about what we got
        for set_name, model_results in results.items():
            print(f"\n{set_name}: {len(model_results)} models tested")
            for model_name, summaries in model_results.items():
                print(f"  {model_name}: {len(summaries)} summaries generated")

        # Analyze results
        print("\n" + "="*60)
        print("ANALYZING RESULTS...")
        results_df = analyze_results(quality_scores)

        # Save results
        results_df.to_csv('pilot_study_results.csv', index=False)
        print(f"\nResults saved! Shape: {results_df.shape}")
        print("\nFirst few rows:")
        print(results_df.head())

    except Exception as e:
        print(f"Error during pilot study: {e}")
        import traceback
        traceback.print_exc()

Starting pilot study...
Loading data...
Setting up models...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

...

Processing multiple with distilbart...


Your max_length is set to 150, but your input_length is only 50. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Your max_length is set to 150, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 150, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_length is set to 150, but your input_length is only 43. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)
Your

Processing multiple with led...


Your max_length is set to 150, but your input_length is only 50. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Your max_length is set to 150, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Yo

Assessing quality...

Got results for 5 test sets
Got quality scores for 5 test sets

clean: 8 models tested
  bart-cnn: 10 summaries generated
  bart-xsum: 10 summaries generated
  t5-base: 10 summaries generated
  t5-large: 10 summaries generated
  pegasus-xsum: 10 summaries generated
  pegasus-cnn: 10 summaries generated
  distilbart: 10 summaries generated
  led: 10 summaries generated

conditionals: 8 models tested
  bart-cnn: 9 summaries generated
  bart-xsum: 9 summaries generated
  t5-base: 9 summaries generated
  t5-large: 9 summaries generated
  pegasus-xsum: 9 summaries generated
  pegasus-cnn: 9 summaries generated
  distilbart: 9 summaries generated
  led: 9 summaries generated

uncertainty: 8 models tested
  bart-cnn: 10 summaries generated
  bart-xsum: 10 summaries generated
  t5-base: 10 summaries generated
  t5-large: 10 summaries generated
  pegasus-xsum: 10 summaries generated
  pegasus-cnn: 10 summaries generated
  distilbart: 10 summaries generated
  led: 10 summar